In [1]:
!pip install sklearn_crfsuite

In [2]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from nltk.corpus import stopwords
import pandas as pd
import pickle
import re
import os

In [3]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
## ADDITIONAL FUNCTIONS
if not os.path.exists('models'):
    os.makedirs('models')
    
pStopWordsList = stopwords.words('english') + stopwords.words('russian')

def SearchGoodsinTxt(ptxt, pGoodsPN, pGoodsDescription, pGoodsQty, pGoodsUnit, spaces=3):
    if (pGoodsPN==None)&(pGoodsDescription==None):
        return None
    if pGoodsPN in (None, ''): 
        pGoodsPN = '' 
    else: 
        pGoodsPN= '%s.{0,%s}' % (re.escape(pGoodsPN.lower()), spaces)
    if pGoodsDescription in (None, ''): 
        pGoodsDescription = '' 
    else: 
        pGoodsDescription= '%s.{0,%s}' % (re.escape(pGoodsDescription.lower()), spaces)
    if pGoodsQty in (None, ''): 
        pGoodsQty = '' 
    else: 
        pGoodsQty= '%s.{0,%s}' % (re.escape(pGoodsQty.lower()), spaces)
        
    if pGoodsUnit in (None, ''): 
        pGoodsUnit = '' 
    else: 
        pGoodsUnit= '%s.{0,%s}' % (re.escape(pGoodsUnit.lower()), spaces)
        
    regext = '%s%s%s%s' % (pGoodsPN, pGoodsDescription, pGoodsQty, pGoodsUnit)
    #print(regext)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsDescription, pGoodsPN, pGoodsQty, pGoodsUnit)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsDescription, pGoodsPN, pGoodsUnit,pGoodsQty)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    regext = '%s%s%s%s' % (pGoodsPN, pGoodsDescription, pGoodsUnit,pGoodsQty)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    regext = '%s%s%s%s' % (pGoodsQty, pGoodsUnit, pGoodsPN, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsUnit, pGoodsQty, pGoodsPN, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsUnit, pGoodsQty, pGoodsDescription,pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsQty, pGoodsUnit, pGoodsDescription,pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsPN, pGoodsQty, pGoodsUnit, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsDescription, pGoodsQty, pGoodsUnit, pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsPN, pGoodsUnit, pGoodsQty, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s%s' % (pGoodsDescription, pGoodsUnit, pGoodsQty, pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
   
    regext = '%s%s%s' % (pGoodsPN, pGoodsDescription, pGoodsQty)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s' % (pGoodsDescription, pGoodsPN, pGoodsQty)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s%s' % (pGoodsQty, pGoodsPN, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    regext = '%s%s%s' % (pGoodsQty, pGoodsDescription, pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    #return m
    regext = '%s%s' % (pGoodsPN, pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s%s' % ( pGoodsDescription, pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]

    regext = '%s' % (pGoodsPN)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    
    regext = '%s' % (pGoodsDescription)
    m = re.search(regext, ptxt)
    if m!=None:
        return m.string[m.start(0):m.end(0)][:-spaces]
    return m


def process_text_layer(TextLayer, pStopWordsList = []):
    txt =  " ".join([w for w in TextLayer.lower().split() \
                     if (not w in pStopWordsList)])

    txt = txt. \
        replace('«', ''). \
        replace('»', ''). \
        replace('(', ''). \
        replace(')', ''). \
        replace('\[)', ''). \
        replace('\]', ''). \
        replace('^', ''). \
        replace('\\', '')

    return txt

def tagging_sentence_by_targets(ptag, sentence_tagged, df_targets, pmodel_name):
    ret = sentence_tagged
    for r in df_targets[df_targets['tag']==ptag]['string_value']:
        tagging_sentence(ret, r, pmodel_name)
    
    return ret

def word2features2(sent, i, exclude = 'O', bias=3):
    word = sent[i][0]
    if sent[i][2] == exclude:
        return None

    features = {
        'bias': bias,
        'word.lower()': word.lower(),
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isdigit()': word.isdigit(),
        'BOS': False,
        'EOS': False
    }
    for w in range(1, bias+1):
        if i > w:
            word1 = sent[i-w][0]
            features.update({
                '-%s:word.lower()' % (w): word1.lower()
            })
            
    if i < 1:
        features['BOS'] = True

    for w in range(1, bias+1):
        if i < len(sent)-w:
            word1 = sent[i+w][0]
            features.update({
                '+%s:word.lower()' % (w): word1.lower()
            })
            
    if i >= len(sent)-1:
        features['EOS'] = True

    return features

def sent2features(sent, exclude = 'O'):
    result = list()
    for i in range(len(sent)):
        ret = word2features2(sent, i, exclude = exclude, bias=10)
        if ret !=None:
            result.append(ret)
    return result

def sent2labels(sent, exclude = 'O'):
    result = list()
    for token, postag, label in sent:
        if label !=exclude:
            result.append(label)        
    return result

def tagging_sentence(sentence, target, tag):
    sentence_text=  ' '.join([w['Word'] for w in sentence])
    r_tag = ''

    try:
        res_find  = re.finditer(target.lower(), sentence_text.lower())
    except:
        return sentence

    for m in res_find:
        r_tag = 'B-%s' % tag
        for i in range(len(sentence_text[:m.start()].split()), len(sentence_text[:m.end()].split())):
            """
            if r_tag==('B-%s' % tag):
                if i>0:
                    sentence[i-1]['Tag']= 'B1-%s' % tag
                if i>1:
                    sentence[i-2]['Tag']= 'B2-%s' % tag                      
                if i>2:
                    sentence[i-3]['Tag']= 'B3-%s' % tag
                if i>3:
                    sentence[i-4]['Tag']= 'B4-%s' % tag  
            """
            sentence[i]['Tag']= r_tag
            r_tag = 'I-%s' % tag   
            """
            for j in range(1,5):
                if i+j < len(sentence):
                    sentence[i+j]['Tag']='E%s-%s' % (str(j), tag)
            """
    return sentence

def prepare_sentence(num, sentence):
    words = sentence.replace('"', '').split()
    tmp = []
    for w in words:
        tmp.append({'Sentence #': 'Sentence: %s' % num, 'Word' : w, 'POS' : 'NN', 'Tag': 'O'})
    return tmp

In [5]:
!gdown --id 1mvcIhS_E4YhOMP3Tcm898aOjVDZIqLp-

Downloading...
From: https://drive.google.com/uc?id=1mvcIhS_E4YhOMP3Tcm898aOjVDZIqLp-
To: /content/NER RFQ.csv
5.41MB [00:00, 77.1MB/s]


In [6]:
data = pd.read_csv('NER RFQ.csv')
for column in data.columns:
    data[column] = data[column].astype(str)

In [7]:
GoodsString = data.copy()
GoodsString['tag'] = GoodsString.apply(lambda r: r['tag'].split('-')[0], axis =1)
GoodsString['string_value'] = GoodsString.apply(lambda r: process_text_layer(r['string_value'], pStopWordsList), axis = 1)

GoodsString['Match'] = GoodsString.apply(lambda r: SearchGoodsinTxt(r['string_value'], r['GoodsPN'], r['GoodsDescription'], r['GoodsQTY'], r['GoodsUnit'], 5), axis=1)

In [8]:
model_name = 'GoodsString'
df_x = GoodsString[GoodsString['Match'].notnull()][['tag', 'string_value']].rename(columns={'string_value':'x'})
df_y = GoodsString[GoodsString['Match'].notnull()][['tag', 'Match']].rename(columns={'Match': 'string_value'})
df_x["x_tagged"] = df_x.apply(lambda r: prepare_sentence(r.name, r["x"]), axis=1)
df_x["x_tagged"] = df_x.apply(lambda r: tagging_sentence_by_targets(r["tag"], r['x_tagged'], df_y, model_name), axis=1)

In [9]:
validation_set_part = 0.2

df_validation = df_x.sample(round(len(df_x)*validation_set_part)).copy()
df_training = df_x[df_x['tag'].isin(df_validation['tag'].values)==False].copy()

In [10]:
df_training.head()

,tag,x,x_tagged
70,119287R.msg,name 119287r.msg <aogdesk@nordstar.ru> <mvf@it...,"[{'Sentence #': 'Sentence: 70', 'Word': 'name'..."
74,119735R.msg,name 119735r.msg <aogdesk@nordstar.ru> <mvf@it...,"[{'Sentence #': 'Sentence: 74', 'Word': 'name'..."
76,120421R.msg,"name 120421r.msg ""d.klebcha@s7.ru"" <d.klebcha@...","[{'Sentence #': 'Sentence: 76', 'Word': 'name'..."
88,120660R.msg,"name 120660r.msg ""yuliya a. kondratova"" <yuliy...","[{'Sentence #': 'Sentence: 88', 'Word': 'name'..."
89,120660R.msg,"name 120660r.msg ""yuliya a. kondratova"" <yuliy...","[{'Sentence #': 'Sentence: 89', 'Word': 'name'..."


In [11]:
df_training["tags"] = df_training.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
df_training["words"] = df_training.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])

In [12]:
df_training = df_training.loc[:, ['tag', 'words', 'tags']].rename(columns={'tag' : 'id', 'tags' : 'ner_tags', 'words' : 'tokens'})
df_training.head()

,id,tokens,ner_tags
70,119287R.msg,"[name, 119287r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
74,119735R.msg,"[name, 119735r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
76,120421R.msg,"[name, 120421r.msg, d.klebcha@s7.ru, <d.klebch...","[O, O, O, O, O, O, O, O, B-GoodsString, I-Good..."
88,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
89,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [13]:
df_validation["tags"] = df_validation.x_tagged.apply(lambda sent : [elem['Tag'] for elem in sent])
df_validation["words"] = df_validation.x_tagged.apply(lambda sent : [elem['Word'] for elem in sent])

In [14]:
df_validation = df_validation.loc[:, ['tag', 'words', 'tags']].rename(columns={'tag' : 'id', 'tags' : 'ner_tags', 'words' : 'tokens'})
df_validation.head()

,id,tokens,ner_tags
1210,124318R.msg,"[name, 124318r.msg, =?koi8-r?b?98zb08xoy88g4cz...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2242,125610R.msg,"[name, 125610r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
672,123514R.msg,"[name, 123514r.msg, natallia.baboryk, <natalli...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1925,125288R.msg,"[name, 125288r.msg, <aogdesk@nordstar.ru>, <rf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
631,123468R.msg,"[name, 123468r.msg, <aog-desk@airbridgecargo.c...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [15]:
print(df_training.shape, df_validation.shape)

(807, 3) (385, 3)


In [16]:
df = pd.concat([df_training, df_validation])
df.head()

,id,tokens,ner_tags
70,119287R.msg,"[name, 119287r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
74,119735R.msg,"[name, 119735r.msg, <aogdesk@nordstar.ru>, <mv...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
76,120421R.msg,"[name, 120421r.msg, d.klebcha@s7.ru, <d.klebch...","[O, O, O, O, O, O, O, O, B-GoodsString, I-Good..."
88,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
89,120660R.msg,"[name, 120660r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [17]:
df.to_csv("NER_RFQ_agg.csv", index=False)

In [18]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)

In [19]:
train.head()

,id,tokens,ner_tags
869,123783R.msg,"[name, 123783r.msg, yuliya, a., kondratova, <y...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1980,125356R.msg,"[name, 125356r.msg, <anton.elenberger@utair.ru...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1916,125271R.msg,"[name, 125271r.msg, <anton.peshko@utair.ru>, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1255,124370R.msg,"[name, 124370r.msg, <gennadiy.moskatov@utair.r...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2264,125625R.msg,"[name, 125625r.msg, <anton.peshko@utair.ru>, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [20]:
test.head()

,id,tokens,ner_tags
771,123670R.msg,"[name, 123670r.msg, =?koi8-r?b?68ht2nhoz9fbiod...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
913,123859R.msg,"[name, 123859r.msg, <logistic@pobeda.aero>, <l...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
948,123906R.msg,"[name, 123906r.msg, <aogdesk@nordstar.ru>, <kk...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
657,123507R.msg,"[name, 123507r.msg, <aogdesk@nordstar.ru>, <rf...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1641,124918R.msg,"[name, 124918r.msg, <anton.peshko@utair.ru>, a...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [21]:
print(train.shape, test.shape)

(953, 3) (239, 3)


In [22]:
train.to_csv('NER_RFQ_agg_train.csv', index=False)
test.to_csv('NER_RFQ_agg_test.csv', index=False)

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!ls

 drive				   NER_RFQ_agg.csv	  'NER RFQ.csv'
 models				   NER_RFQ_agg_test.csv    sample_data
'models\NER_RFQ_GoodsString.crf'   NER_RFQ_agg_train.csv


In [ ]:
!mv NER_RFQ_agg.csv drive/MyDrive/NER_RFQ_agg.csv
!mv NER_RFQ_agg_test.csv drive/MyDrive/NER_RFQ_agg_test.csv
!mv NER_RFQ_agg_train.csv drive/MyDrive/NER_RFQ_agg_train.csv